<a href="https://colab.research.google.com/github/ginttone/multi_deeplearning/blob/master/Naver_SentimentAnalysis_loadmodel_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

업무분장시 딥러닝에서 저장한 .h5 파일을 웹서버가 받아서 사용 

In [1]:
import tensorflow as tf

In [2]:
loaded_model = tf.keras.models.load_model('./NaverSentimentAnalysis_LSTM.h5')
loaded_model

In [7]:
# !python -m pip install konlpy

`>>>`불용어 리스트를 픽클로 저장해서 가져옴

In [9]:
import pickle

stopwords = pickle.load(open('./stopwords.pkl','rb'))

In [4]:
import konlpy

감독이 예술이네 : 측정하고싶은 것 (웹사용자 입력)



In [5]:
okt=konlpy.tag.Okt()

In [6]:
new_sentence = okt.morphs('감독이 예술이네', stem=True)

기존 불용어 처리
```
  words = list()
  for tok in temp_x:
    if tok not in stopwords:
      words.append(tok)
```

많이 쓰이는 불용어 처리(한 줄로 표현)

```
words의 성격이 리스트 이기때문에
[문장으로 넣음]

words.append(tok) for tok in temp_x if tok not in stopwords
```

In [10]:
new_sentence = [tok for tok in new_sentence if tok not in stopwords]

In [11]:
new_sentence

['감독', '예술', '이네']

`>>>` tokenizer 가져오기

In [14]:
tokenizer = pickle.load(open('./tokenizer.pkl','rb'))
tokenizer

In [19]:
# tokenizer.word_counts
# tokenizer.word_index
vocb_size = len(tokenizer.word_index)
vocb_size

1572

## padding

자연어를 정재한 것만 뺀 것 가지고 숫자화 시키기

In [22]:
encoded = tokenizer.texts_to_sequences([new_sentence])
# encoded

pad_new = tf.keras.preprocessing.sequence.pad_sequences(encoded, maxlen=50)
pad_new

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,  92, 128]],
      dtype=int32)

## predict

In [25]:
score = loaded_model.predict(pad_new)
score

array([[0.49929982]], dtype=float32)

##Service

결과 보고 긍정 , 부정 판별해서 서비스해주기

In [26]:
if(score>0.5):
  print('긍정', score)
else:
  print('부정', score)

부정 [[0.49929982]]


`+++`펑션만들기

```
okt=konlpy.tag.Okt()
new_sentence = okt.morphs('감독이 예술이네', stem=True)
new_sentence = [tok for tok in new_sentence if tok not in stopwords]

encoded = tokenizer.texts_to_sequences([new_sentence])

pad_new = tf.keras.preprocessing.sequence.pad_sequences(encoded, maxlen=50)

score = loaded_model.predict(pad_new)
```

```
def fname():
  return
```

In [27]:
def sentiment_predict(sentence):
  okt=konlpy.tag.Okt()
  new_sentence = okt.morphs(sentence, stem=True)
  new_sentence = [tok for tok in new_sentence if tok not in stopwords]
  encoded = tokenizer.texts_to_sequences([new_sentence])
  pad_new = tf.keras.preprocessing.sequence.pad_sequences(encoded, maxlen=50)
  score = loaded_model.predict(pad_new)
  return score

In [30]:
words = '이 영화 꽝이네..'

sentiment_predict(words)

array([[0.50130105]], dtype=float32)

In [31]:
words = '영화가 좋으니 마음도 좋네요'

sentiment_predict(words)

array([[0.5108125]], dtype=float32)